## Задача 4-1. Линейное программирование: формулирование задачи.

В этой задаче Вам предлагается закодировать задачу TSP для заданного графа в виде задачи ЦЛП с помощью условий MTZ (Миллера—Таккера—Землина) и поработать с библиотекой [PuLP](https://pypi.python.org/pypi/PuLP/1.6.5). Если Вы используете дистрибутив Anaconda, то эта библиотека не находится утилитой conda, зато совершенно нормально устанавливается с помощью pip:

`pip install pulp`

Вам дана функция `dist15`, которая по двум номерам вершин возвращает вес ребра между ними. Номера от 1 до 15, пример взят [отсюда](https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html).
Нужно построить соответствующую задачу ЦЛП и решить её средствами PuLP, чтобы найти оптимальный гамильтонов цикл в графе. Сделать это следует в функции `solve_tsp_with_lp`, которая получает на вход размерность задачи и весовую функцию, а на выходе даёт перестановку номеров вершин графа (нумеруемых с единицы), соответствующую оптимальному гамильтонову циклу.

In [41]:
from pulp import LpVariable, LpProblem, LpConstraint, LpMaximize, LpMinimize, lpSum

In [42]:
def dist15(i: int, j: int) -> int:
    return (
        list(
            map(
                int, 
                filter(
                    lambda s: len(s.strip()) > 0, '''
                    0 29 82 46 68 52 72 42 51 55 29 74 23 72 46 
                    29  0 55 46 42 43 43 23 23 31 41 51 11 52 21 
                    82 55  0 68 46 55 23 43 41 29 79 21 64 31 51 
                    46 46 68  0 82 15 72 31 62 42 21 51 51 43 64 
                    68 42 46 82  0 74 23 52 21 46 82 58 46 65 23 
                    52 43 55 15 74  0 61 23 55 31 33 37 51 29 59 
                    72 43 23 72 23 61  0 42 23 31 77 37 51 46 33 
                    42 23 43 31 52 23 42  0 33 15 37 33 33 31 37 
                    51 23 41 62 21 55 23 33  0 29 62 46 29 51 11 
                    55 31 29 42 46 31 31 15 29  0 51 21 41 23 37 
                    29 41 79 21 82 33 77 37 62 51  0 65 42 59 61 
                    74 51 21 51 58 37 37 33 46 21 65  0 61 11 55 
                    23 11 64 51 46 51 51 33 29 41 42 61  0 62 23 
                    72 52 31 43 65 29 46 31 51 23 59 11 62  0 59 
                    46 21 51 64 23 59 33 37 11 37 61 55 23 59  0
                    '''.split()
                )
            )
        )[(i-1) * 15 + (j-1)]
    )

In [43]:
from typing import List, Callable

In [68]:
def solve_tsp_with_lp(num_vertices: int, distance_function: Callable[[int, int], int]) -> List[int]:
    
    lp = LpProblem(name='TSProblem with MTZ', sense=LpMinimize)
    
    edges = [(i, j) for i in range(num_vertices) 
             for j in range(num_vertices) if (i != j)]
    
    e_ind = []
    for i in range(num_vertices):
        e_ind.append([])
        for j in range(num_vertices):
            e_ind[i].append(LpVariable(name='{} {}'.format(i, j), lowBound=0, upBound=1, cat='Integer'))
    
    
                            
    #                             
                                
    lp.objective = lpSum(e_ind[i][j] * distance_function(i+1, j+1) for i, j in edges)
    
    for i in range(num_vertices):
        lp += (lpSum(e_ind[i][j] for j in range(num_vertices)) == 1)
        lp += (lpSum(e_ind[j][i] for j in range(num_vertices)) == 1)
        
        lp += (e_ind[i][i] == 0)
    
    
    u = [LpVariable(name='u{}'.format(i), lowBound=0, upBound=num_vertices, cat='Integer') for i in range(num_vertices)]
        
    for i, j in edges:
        if i != 1:
            lp += (u[i] - u[j] + num_vertices * e_ind[i][j] <= num_vertices - 1)
            
    lp_status = lp.solve()
    
    res = []
    res.append(1)
    
    
        
    tmp = 0
    for i in range(num_vertices):
        for j in range(num_vertices):
            if e_ind[tmp][j].value() == 1:
                tmp = j
                break
        res.append(tmp + 1)    
            
          
              
    
    print(lp.objective.value())
    return res

print(' ⟶ '.join(map(str, solve_tsp_with_lp(15, dist15))))

291.0
1 ⟶ 11 ⟶ 4 ⟶ 6 ⟶ 8 ⟶ 10 ⟶ 14 ⟶ 12 ⟶ 3 ⟶ 7 ⟶ 5 ⟶ 9 ⟶ 15 ⟶ 2 ⟶ 13 ⟶ 1
